In [1]:
# reading in data
! pip install kaggle 

Defaulting to user installation because normal site-packages is not writeable
  Preparing metadata (setup.py) ... done
  Created wheel for kaggle: filename=kaggle-1.6.17-py3-none-any.whl size=105803 sha256=fa644eff027b14715a67f238539ade63dd06f8870abf11792a6314e4a20eb051
  Stored in directory: /Users/carliestewart/Library/Caches/pip/wheels/2b/af/a9/70bffa2773af622d2ebea9c8d407720b86e67bd40c465bf837
Successfully built kaggle


In [4]:
# reading in the olympics data
! kaggle datasets download -d mohamedyosef101/2024-olympics-medals-and-economic-status
! unzip /Users/carliestewart/Documents/DS2002-Data-Project-1/2024-olympics-medals-and-economic-status.zip -d .

/Users/carliestewart/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(
Dataset URL: https://www.kaggle.com/datasets/mohamedyosef101/2024-olympics-medals-and-economic-status
License(s): MIT
  0%|                                               | 0.00/2.08k [00:00<?, ?B/s]
100%|██████████████████████████████████████| 2.08k/2.08k [00:00<00:00, 1.00MB/s]


In [7]:
# reading in the cricket json 
! kaggle datasets download -d diishasiing/cricket-analysis-dataset
! unzip /Users/carliestewart/Documents/DS2002-Data-Project-1/cricket-analysis-dataset.zip -d .

/Users/carliestewart/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(
Dataset URL: https://www.kaggle.com/datasets/diishasiing/cricket-analysis-dataset
License(s): CC0-1.0
cricket-analysis-dataset.zip: Skipping, found more recently modified local copy (use --force to force download)
Archive:  /Users/carliestewart/Documents/DS2002-Data-Project-1/cricket-analysis-dataset.zip
  inflating: ./df_batting.csv        
  inflating: ./df_bowling.csv        
  inflating: ./df_match.csv          
  inflating: ./df_players.csv        
  inflating: ./t20_wc_batting_summary.json  
  inflating: ./t20_wc_bowling_summary.json  
  inflating: ./t20_wc_match_results.json  
  inflating: ./t20_wc_player_info.json  


In [17]:
import pandas as pd
olympics_csv = pd.read_csv('olympics-economics.csv')
olympics_csv.head()


olympics_num_rows = len(olympics_csv)
print("Number of rows: " + str(olympics_num_rows))
olympics_num_cols = len(olympics_csv.columns)
print("Number of columns: " + str(olympics_num_cols))


Number of rows: 90
Number of columns: 9


In [19]:
cricket_json = pd.read_json('t20_wc_batting_summary.json')
cricket_num_rows = len(cricket_json)
print("Number of rows: " + str(cricket_num_rows))

cricket_num_cols = len(cricket_json.columns)
print("Number of columns: " + str(cricket_num_cols))


Number of rows: 45
Number of columns: 1
